# Visão geral do SYCL

##### Seções
- [O que é Data Parallel C++?](#O-que-é-Data-Parallel-C++?)
- _Código:_ [Seletor de dispositivo](#Seletor-de-dispositivo)
- [Kernels paralelos](#Kernels-paralelos)
- [Anatomia do código SYCL](#Anatomia-do-código-SYCL)
- _Código:_ [Dependência implícita com acessores (accessors)](#Dependência-implícita-com-acessores-(accessors))
- _Código:_ [Sincronização: Host Accessor](#Sincronização:-Host-Accessor)
- _Código:_ [Sincronização: Destruição do _buffer_](#Sincronização:-Destruição-do-buffer)
- _Código:_ [Seletor de dispositivo personalizado](#Seletor-de-dispositivo-personalizado)
- _Código:_ [Multiplicação de números complexos](#Exercício:-multiplicação-de-números-complexos)

## Objetivos de aprendizado
* Explicar as classes fundamentais do __SYCL__
* Usar __device selection__ para descarregar cargas de trabalho do kernel
* Decidir quando usar __basic parallel kernels__ e __ND Range Kernels__
* Criar um __host Accessor__
* Criar uma aplicação __SYCL__ por meio de exercícios práticos de laboratório

***
## O que é Data Parallel C++?
Os programas __oneAPI__ são escritos em __Data Parallel C++ (DPC++)__. Ele é baseado em benefícios de produtividade do C++ moderno e construções familiares e incorpora o padrão __SYCL__ para paralelismo de dados e programação heterogênea. DPC++ é uma __single source__ onde __host code__ e __heterogeneous accelerator kernels__ podem ser misturados nos mesmos arquivos fonte. Um programa DPC++ é chamado no computador _host_ e transfere a computação para um acelerador. Os programadores usam C++ familiar e construções de biblioteca com funcionalidades adicionais, como __queue__ para direcionamento de trabalho, __buffer__ para gerenciamento de dados e __parallel_for__ para paralelismo para direcionar quais partes da computação e dados devem ser descarregados.

## Dispositivo
A classe __device__ representa os recursos dos aceleradores em um sistema que utiliza Intel&reg; oneAPI Toolkits. A classe de dispositivo contém funções de membro para consultar informações sobre o dispositivo, o que é útil para programas SYCL onde vários dispositivos são criados.
* A função __get_info__ fornece informações sobre o dispositivo:
  * Nome, fornecedor e versão do dispositivo
  * Os IDs de item de trabalho local e global
  * Largura para tipos integrados, frequência de clock, largura e tamanhos de cache, online ou offline
 
```cpp
queue q;
device my_device = q.get_device();
std::cout << "Device: " << my_device.get_info<info::device::name>() << std::endl;
```

***
## Seletor de dispositivo
A classe __device_selector__ permite a seleção em tempo de execução de um dispositivo específico para executar _kernels_ com base em heurísticas fornecidas pelo usuário. O exemplo de código a seguir mostra o uso dos seletores de dispositivo padrão (__default_selector, cpu_selector, gpu_selector…__) e um device_selector derivado

 
```cpp
default_selector selector;
// host_selector selector;
// cpu_selector selector;
// gpu_selector selector;
queue q(selector);
std::cout << "Device: " << q.get_device().get_info<info::device::name>() << std::endl;
```

O código SYCL abaixo mostra diferentes seletores de dispositivo: Inspecione o código, não há modificações necessárias:
1. Inspecione a célula de código abaixo e clique em executar ▶ para salvar o código no arquivo
2. Em seguida, execute ▶ a célula na seção __Build and Run__ abaixo do código para compilar e executar o código.

In [ ]:
%%writefile lab/gpu_sample.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>

using namespace cl::sycl;

int main() {
  //# Create a device queue with device selector
  
  gpu_selector selector;
  //cpu_selector selector;
  //default_selector selector;
  //host_selector selector;
  
  queue q(selector);

  //# Print the device name
  std::cout << "Device: " << q.get_device().get_info<info::device::name>() << std::endl;

  return 0;
}

### Compilar e Executar
Selecione a célula abaixo e clique em executar ▶ para compilar e executar o código:

(Como você está executando em uma GPU, os scripts são necessários para executar em um nó com esse hardware.)

In [ ]:
! chmod 755 q; chmod 755 run_gpu.sh;if [ -x "$(command -v qsub)" ]; then ./q run_gpu.sh; else ./run_gpu.sh; fi

Se as células do Jupyter não responderem ou se apresentarem um erro ao compilar os exemplos de código, reinicie o kernel do Jupyter. Selecione o seguinte item de menu na janela Jupyter: `Kernel > Restart Kernel and Clear All Outputs` e compile os exemplos de código novamente.

## Fila (Queue)
__Queue__ submete grupos de comandos a serem executados pelo _runtime_ SYCL. A fila é um mecanismo em que __trabalho é submetido__ a um dispositivo. Uma fila mapeia para um dispositivo e várias filas podem ser mapeadas para o mesmo dispositivo.
 
```cpp
q.submit([&](handler& h) {
    //COMMAND GROUP CODE
});
```

## Kernel
A classe __kernel__ encapsula métodos e dados para a execução de código no dispositivo quando um grupo de comando é instanciado. O objeto do kernel não é explicitamente construído pelo usuário e é construído quando uma função de envio do kernel, como __parallel_for__, é chamada
```cpp
q.submit([&](handler& h) {
  h.parallel_for(range<1>(N), [=](id<1> i) {
    A[i] = B[i] + C[i]);
  });
});
```

## Escolhendo onde os kernels do dispositivo rodam

O trabalho é submetido a filas e cada fila é associada a exatamente um dispositivo (por exemplo, uma GPU ou FPGA específico). Você pode decidir a qual dispositivo uma fila está associada (se desejar) e ter quantas filas desejar para despachar o trabalho em sistemas heterogêneos.

| Dispositivo de destino | Fila |
| ----- | ------- |
| Crie uma fila destinada a qualquer dispositivo: | queue() |
| Crie uma fila destinada a classes pré-configuradas de dispositivos: | queue(cpu_selector{}); queue(gpu_selector{});  queue(intel::fpga_selector{}); queue(accelerator_selector{}); queue(host_selector{});|
| Crie um dispositivo específico de destino de fila (critérios personalizados): | class custom_selector : public device_selector {int operator()(…  // Any logic you want! … queue(custom_selector{}); |

<center><img src="Assets/queue.png"></center>

## Linguagem DPC++ e _runtime_
A linguagem e o _runtime_ DPC++ consistem em um conjunto de classes, modelos e bibliotecas C++.

__Escopo de aplicação__ e __escopo de grupo de comandos__:
  * Código que executa no host
  * Os recursos completos do C++ estão disponíveis no escopo da aplicação e do grupo de comandos

__Escopo do Kernel__:
  * Código que executa no dispositivo
  * No __Escopo do Kernel__, existem __limitações__ no C++ aceito

***
## Kernels paralelos

__Kernels paralelos__ permitem que várias instâncias de uma operação sejam executadas em paralelo. Isso é útil para __descarregar (offload)__ a execução paralela de um __for-loop__ básico no qual cada iteração é completamente independente e em qualquer ordem. Kernels paralelos são expressos usando a função __parallel_for__
Um simples loop 'for' em um aplicativo C++ é escrito como abaixo

```cpp
for(int i=0; i < 1024; i++){
    a[i] = b[i] + c[i];
});
```

Abaixo está como você pode descarregar para o acelerador

```cpp
h.parallel_for(range<1>(1024), [=](id<1> i){
    A[i] =  B[i] + C[i];
});
```

## Kernels Paralelos Básicos

A funcionalidade dos Kernels Paralelos Básicos é exposta por meio das classes __range__, __id__ e __item__. A classe __range__ é usada para descrever o __espaço de iteração__ da execução paralela e a classe __id__ é usada para __índice__ uma instância individual de um kernel em uma execução paralela


```cpp
h.parallel_for(range<1>(1024), [=](id<1> i){
    // CODE THAT RUNS ON DEVICE 
    
});

```

O exemplo acima é suficiente se tudo o que você precisa é o __índice(id)__, mas se você precisa do __intervalo (range)__ em seu código de kernel, então você pode usar a classe __item__ em vez da classe __id__, que você pode usar para consultar o __range__ como mostrado abaixo. A classe __item__ representa uma __instância individual__ de uma função do kernel, expõe funções adicionais para propriedades de consulta do intervalo de execução


```cpp
h.parallel_for(range<1>(1024), [=](item<1> item){
    auto i = item.get_id();
    auto R = item.get_range();
    // CODE THAT RUNS ON DEVICE
    
});
```

## Kernels NDRange
Os Kernels Paralelos Básicos são uma maneira fácil de paralelizar um loop `for`, mas não permitem a otimização do desempenho no nível do hardware. O __Kernel NDRange__ é outra maneira de expressar paralelismo que permite ajuste de desempenho de baixo nível, fornecendo acesso à __memória local e mapeamento de execuções__ para unidades de computação no hardware. Todo o espaço de iteração é dividido em grupos menores chamados __work-groups__, __work-items__ dentro de um __work-groups__ são agendados em uma única unidade de computação no hardware.

O agrupamento de execuções do _kernel_ em __work-groups__ permite o controle do uso de recursos e de balanceamento de carga na distribuição de trabalho. A funcionalidade dos kernels __NDRange__ é exposta por meio das classes __nd_range__ e __nd_item__. A classe __nd_range__ representa um intervalo de execução agrupado (__grouped execution range__) usando o intervalo de execução global e o intervalo de execução local de cada grupo de trabalho. A classe __nd_item__ representa uma __instância individual__ de uma função do _kernel_ e permite consultar o intervalo e o índice do grupo de trabalho.

```cpp
h.parallel_for(nd_range<1>(range<1>(1024),range<1>(64)), [=](nd_item<1> item){
    auto idx = item.get_global_id();
    auto local_id = item.get_local_id();
    // CODE THAT RUNS ON DEVICE
});
```
<center><img src="Assets/ndrange.png"></center>

***
## Modelo de Buffer 
__Buffers__ encapsulam dados em uma aplicação SYCL em ambos dispositivo e host. __Accessors__ é o mecanismo para acessar os dados do _buffer_.

### Anatomia do código SYCL
Os programas que utilizam SYCL requerem a inclusão do cabeçalho __cl/sycl.hpp__. Recomenda-se empregar a instrução de namespace para evitar a digitação de referências repetidas do namespace `cl::sycl`.

```cpp
#include <CL/sycl.hpp>
using namespace cl::sycl;
```

__Programas SYCL__ são C++ padrão. O programa é invocado no computador __host__ e transfere a computação para o __accelerador__. Um programador usa __queue, buffer, dispositivo e abstrações de kernel__ do SYCL para direcionar quais partes da computação e dados devem ser descarregados.

Como primeiro passo em um programa SYCL, criamos uma __queue__. Descarregamos a computação para um __device__ enviando tarefas para uma fila. O programador pode escolher CPU, GPU, FPGA e outros dispositivos por meio do __selector__. Este programa usa o padrão aqui, o que significa que o _runtime_ SYCL seleciona o dispositivo mais capaz disponível em tempo de execução usando o seletor padrão. Falaremos sobre os dispositivos, seletores de dispositivo e os conceitos de buffers, acessores e kernels nos próximos módulos, mas abaixo está um programa SYCL simples para você começar com os conceitos acima.

O dispositivo e o host podem compartilhar __memória física__ ou ter memórias distintas. Quando as memórias são distintas, o descarregamento de computação requer __copia de dados entre o host e o dispositivo__. SYCL não requer que o programador gerencie as cópias dos dados. Ao criar __Buffers e Accessors__, o SYCL garante que os dados estejam disponíveis para o host e o dispositivo sem nenhum esforço do programador. O SYCL também permite ao programador controle explícito sobre a movimentação de dados quando é necessário obter o melhor desempenho.

Em um programa SYCL, definimos um __kernel__, que é aplicado a cada ponto em um espaço de índice. Para programas simples como este, o espaço de índice mapeia diretamente para os elementos do arranjo. O _kernel_ é encapsulado em uma __função lambda C++__. A função _lambda_ recebe um ponto no espaço de índice como uma matriz de coordenadas. Para este programa simples, a coordenada do espaço de índice é a mesma que o índice da matriz. O __parallel_for__ no programa abaixo aplica o _lambda_ ao espaço do índice. O espaço de índice é definido no primeiro argumento de `parallel_for` como um __intervalo unidimensional de 0 a N-1__.


O código abaixo mostra a adição de vetor simples usando SYCL. Leia os comentários abordados na etapa 1 à etapa 6.

```cpp
void dpcpp_code(int* a, int* b, int* c, int N) {
  //Etapa 1: criar uma fila de dispositivos
  //(o desenvolvedor pode especificar um tipo de dispositivo por meio do seletor de dispositivo ou usar o seletor padrão)
  queue q;
  //Etapa 2: criar buffers (representa tanto a memória do host quanto a do dispositivo)
  buffer<int,1> buf_a(a, range<1>(N));
  buffer<int,1> buf_b(b, range<1>(N));
  buffer<int,1> buf_c(c, range<1>(N));
  //Etapa 3: enviar um comando para execução (assíncrona)
  q.submit([&](handler &h){
  //Etapa 4: crie acessores (acessors) de buffer para acessar os dados do buffer no dispositivo
  auto A = buf_a.get_access<access::mode::read>(h);
  auto B = buf_b.get_access<access::mode::read>(h);
  auto C = buf_c.get_access<access::mode::write>(h);
  //Etapa 5: enviar um kernel (lambda) para execução
  h.parallel_for(range<1>(N), [=](item<1> i){
    //Etapa 6: escrever um kernel
    //As invocações do kernel são executadas em paralelo
    //I kernel é invocado para cada elemento do intervalo
    //A invocação do kernel tem acesso ao id de invocação
    C[i] = A[i] + B[i];
    });
  });
}
```

## Dependência implícita com acessores (_accessors_)
* Acessores criam __dependências de dados__ no gráfico SYCL que ordenam as execuções do _kernel_ 
* Se dois _kernels_ usam o mesmo _buffer_, o segundo _kernel_ precisa aguardar a conclusão do primeiro _kernel_ para evitar condições de corrida.

<center><img src="Assets/buffer1.png"></center>

O código SYCL abaixo demonstra dependência implícita com acessores: Inspecione o código, não há modificações necessárias:

1. Inspecione a célula de código abaixo e clique em executar ▶ para salvar o código no arquivo

2. Em seguida, execute ▶ a célula na seção __Build and Run__ abaixo do código para compilar e executar o código.

In [ ]:
%%writefile lab/buffer_sample.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>

constexpr int num=16;
using namespace sycl;

int main() {
  auto R = range<1>{ num };
  //Create Buffers A and B
  buffer<int> A{ R }, B{ R };
  //Create a device queue
  queue Q;
  //Submit Kernel 1
  Q.submit([&](handler& h) {
    //Accessor for buffer A
    auto out = A.get_access<access::mode::write>(h);
    h.parallel_for(R, [=](id<1> idx) {
      out[idx] = idx[0]; }); });
  //Submit Kernel 2
  Q.submit([&](handler& h) {
    //This task will wait till the first queue is complete
    auto out = A.get_access<access::mode::write>(h);
    h.parallel_for(R, [=](id<1> idx) {
      out[idx] += idx[0]; }); });
  //Submit Kernel 3
  Q.submit([&](handler& h) { 
    //Accessor for Buffer B
    auto out = B.get_access<access::mode::write>(h);
    h.parallel_for(R, [=](id<1> idx) {
      out[idx] = idx[0]; }); });
  //Submit task 4
  Q.submit([&](handler& h) {
   //This task will wait till kernel 2 and 3 are complete
   auto in = A.get_access<access::mode::read>(h);
   auto inout =
    B.get_access<access::mode::read_write>(h);
  h.parallel_for(R, [=](id<1> idx) {
    inout[idx] *= in[idx]; }); }); 
      
 // And the following is back to device code
 auto result =
    B.get_access<access::mode::read>();
  for (int i=0; i<num; i++)
    std::cout << "result[" << i << "]=" << result[i] << "\n";      
  return 0;
}

### Compilar e Executar
Selecione a célula abaixo e clique em executar ▶ para compilar e executar o código:

In [ ]:
! dpcpp lab/buffer_sample.cpp -o bin/buffer_sample
! bin/buffer_sample

Se as células do Jupyter não responderem ou se apresentarem um erro ao compilar os exemplos de código, reinicie o kernel do Jupyter. Selecione o seguinte item de menu na janela Jupyter: `Kernel > Restart Kernel and Clear All Outputs` e compile os exemplos de código novamente.

## Acessores do host
O acessor de host é aquele que usa o destino de acesso do _buffer_ do host. Ele é criado fora do escopo do grupo de comando e os dados aos quais ele dá acesso estarão disponíveis no host. Eles são usados para sincronizar os dados de volta ao host, construindo os objetos de acesso do host. A destruição do _buffer_ é a outra maneira de sincronizar os dados de volta ao host.

## Sincronização: Host Accessor

O buffer assume a propriedade dos dados armazenados no vetor. A criação do acessor de host é uma chamada __blocante__ e só retornará depois que todos os _kernels_ SYCL enfileirados que modificam o mesmo _buffer_ em qualquer fila concluam a execução e os dados estejam disponíveis para o host por meio desse acessor de host.

O código SYCL abaixo demonstra a sincronização com o acessor de host: inspecione o código, não há modificações necessárias:

1. Inspecione a célula de código abaixo e clique em executar ▶ para salvar o código no arquivo

2. Em seguida, execute ▶ a célula na seção __Build and Run__ abaixo do código para compilar e executar o código.

In [ ]:
%%writefile lab/host_accessor_sample.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <CL/sycl.hpp>
using namespace sycl;

int main() {
    constexpr int N = 16;
    auto R = range<1>(N);
    std::vector<double> v(N, 10);
    queue q;
    // O buffer assume a propriedade dos dados armazenados no vetor
    buffer<double, 1> buf(v.data(), R);
    q.submit([&](handler& h) {
        auto a = buf.get_access<access::mode::read_write>(h);
        h.parallel_for(R, [=](id<1> i) { a[i] -= 2; });
    });
    // A criação do acessor de host é uma chamada blocante e só retornará depois que 
    // todos os kernels syCL enfileirados que modificam o mesmo buffer em qualquer fila 
    // concluírem a execução e os dados estiverem disponíveis para o host.
    auto b = buf.get_access<access::mode::read>();
    for (int i = 0; i < N; i++) 
        std::cout << v[i] << "\n";
    return 0;
}

### Compilar e Executar
Selecione a célula abaixo e clique em executar ▶ para compilar e executar o código:

In [ ]:
! dpcpp lab/host_accessor_sample.cpp -o bin/host_accessor_sample
! bin/host_accessor_sample

Se as células do Jupyter não responderem ou se apresentarem um erro ao compilar os exemplos de código, reinicie o kernel do Jupyter. Selecione o seguinte item de menu na janela Jupyter: `Kernel > Restart Kernel and Clear All Outputs` e compile os exemplos de código novamente.

## Sincronização: Destruição do _buffer_
No exemplo abaixo, a criação do _buffer_ ocorre dentro de um escopo de função separado. Quando a execução avança além desse __escopo de função__, o destruidor de _buffer_ é invocado, cedendo a propriedade dos dados e copiando os dados de volta para a memória do host.

O código SYCL abaixo demonstra a sincronização com a destruição do buffer: inspecione o código, não há modificações necessárias:

1. Inspecione a célula de código abaixo e clique em executar ▶ para salvar o código em um arquivo.

2. Em seguida, execute ▶ a célula na seção __Build and Run__ abaixo do código para compilar e executar o código.

In [ ]:
%%writefile lab/buffer_destruction2.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <CL/sycl.hpp>
constexpr int N = 16;
using namespace sycl;

// A criação do buffer ocorre dentro de um escopo de função separado.
void dpcpp_code(std::vector<double> &v, queue &q) {
    auto R = range<1>(N);
    buffer<double, 1> buf(v.data(), R);
    q.submit([&](handler &h) {
    auto a = buf.get_access<access::mode::read_write>(h);
        h.parallel_for(R, [=](id<1> i) { a[i] -= 2; });
    });
}
int main() {
    std::vector<double> v(N, 10);
    queue q;
    dpcpp_code(v, q);
    // Quando a execução avança além desse escopo de função, 
    // o destruidor de buffer é invocado, cedendo a propriedade
    // dos dados e os copia de volta para a memória do host.
    for (int i = 0; i < N; i++) 
        std::cout << v[i] << "\n";
    return 0;
}

### Compilar e Executar
Selecione a célula abaixo e clique em executar ▶ para compilar e executar o código:

In [ ]:
! dpcpp lab/buffer_destruction2.cpp -o bin/buffer_destruction2
! bin/buffer_destruction2

Se as células do Jupyter não responderem ou se apresentarem um erro ao compilar os exemplos de código, reinicie o kernel do Jupyter. Selecione o seguinte item de menu na janela Jupyter: `Kernel > Restart Kernel and Clear All Outputs` e compile os exemplos de código novamente.

## Seletor de dispositivo personalizado
O código a seguir mostra um `device_selector` que emprega uma heurística de seletor de dispositivo. O dispositivo selecionado prioriza um dispositivo GPU porque a classificação de número inteiro retornada é mais alta do que para CPU ou outro acelerador.

O código SYCL abaixo demonstra o seletor de dispositivo personalizado: Inspecione o código, não há modificações necessárias:

1. Inspecione a célula de código abaixo e clique em executar ▶ para salvar o código em um arquivo.

2. Em seguida, execute ▶ a célula na seção __Build and Run__ abaixo do código para compilar e executar o código.

In [ ]:
%%writefile lab/custom_device_sample.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>
#include <iostream>
using namespace sycl;
class my_device_selector : public device_selector {
public:
    my_device_selector(std::string vendorName) : vendorName_(vendorName){};
    int operator()(const device& dev) const override {
    int rating = 0;
    //Estamos pesquisando o dispositivo personalizado específico de um fornecedor e, 
    //se for um dispositivo GPU, estamos dando a classificação mais alta, 3. 
    //A segunda preferência é dada a qualquer dispositivo GPU 
    //e a terceira preferência é dada a dispositivo CPU.
    if (dev.is_gpu() & (dev.get_info<info::device::name>().find(vendorName_) != std::string::npos))
        rating = 3;
    else if (dev.is_gpu()) rating = 2;
    else if (dev.is_cpu()) rating = 1;
    return rating;
    };
    
private:
    std::string vendorName_;
};
int main() {
    //passe o nome do fornecedor para o qual o dispositivo que você deseja consultar
    std::string vendor_name = "Intel";
    //std::string vendor_name = "AMD";
    //std::string vendor_name = "Nvidia";
    my_device_selector selector(vendor_name);
    queue q(selector);
    std::cout << "Device: "
    << q.get_device().get_info<info::device::name>() << std::endl;
    return 0;
}

### Compilar e Executar
Selecione a célula abaixo e clique em executar ▶ para compilar e executar o código:

(Uma vez que é codificado para preferir uma GPU, os scripts são necessários para executar em um nó com uma GPU.)

In [ ]:
! chmod 755 q; chmod 755 run_custom_device.sh;if [ -x "$(command -v qsub)" ]; then ./q run_custom_device.sh; else ./run_custom_device.sh; fi

Se as células do Jupyter não responderem ou se apresentarem um erro ao compilar os exemplos de código, reinicie o kernel do Jupyter. Selecione o seguinte item de menu na janela Jupyter: `Kernel->Restart Kernel and Clear All Outputs` e compile os exemplos de código novamente.

# Exercício: multiplicação de números complexos
A seguir está a definição de um tipo de classe personalizada que representa números complexos.
* O arquivo [Complex.hpp](./src/Complex.hpp) define a classe `Complex2`.
* A classe `Complex2` tem duas variáveis de membro "real" e "imag" do tipo int.
* A classe `Complex2` tem uma função-membro para realizar a multiplicação de números complexos. A função `complex_mul` retorna o objeto do tipo `Complex2` realizando a multiplicação de dois números complexos.
* Vamos chamar a função `complex_mul` de nosso código SYCL.

## EXERCÍCIO
* Neste laboratório, fornecemos o código-fonte que calcula a multiplicação de dois números complexos, onde a classe `Complex2` é a definição de um tipo personalizado que representa números complexos.
* Neste exemplo, o aluno aprenderá como usar o seletor de dispositivo personalizado para direcionar a GPU ou CPU de um fornecedor específico e, em seguida, passar um vetor de objetos de classe complexos personalizados em paralelo. O aluno precisa modificar o código-fonte para selecionar GPU Intel® como a primeira escolha e, em seguida, configure um acessor de gravação e chame a função de membro da classe `Complex2` como kernel para calcular a multiplicação.
* Siga a __Etapa 1 e a Etapa 2 e a Etapa 3 no código abaixo__.
* A classe `Complex2` no exemplo abaixo é para demonstrar o uso de uma classe customizada e como uma classe customizada pode ser passada em um código SYCL, mas não para mostrar a funcionalidade da classe complexa na biblioteca `std`. Você pode usar a própria biblioteca `std::complex` em um programa SYCL.

1. Selecione a célula de código abaixo, __siga as Etapas 1 a 3__ nos comentários do código, clique em executar ▶ para salvar o código em arquivo.
2. Em seguida, execute ▶ a célula na seção __Build and Run__ abaixo do código para compilar e executar o código.

In [ ]:
%%writefile lab/complex_mult.cpp

//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>
#include <iomanip>
#include <vector>
#include "Complex.hpp"

using namespace sycl;
using namespace std;

// Quantidade de números complexos passando para o código SYCL
static const int num_elements = 100;

class custom_device_selector : public device_selector {
public:
    custom_device_selector(std::string vendorName) : vendorName_(vendorName){};
    int operator()(const device& dev) const override {
    int rating = 0;    
    //Estamos pesquisando o dispositivo personalizado específico de um fornecedor e, 
    //se for um dispositivo GPU, estamos dando a classificação mais alta, 3. 
    //A segunda preferência é dada a qualquer dispositivo GPU e 
    //a terceira preferência é dada a dispositivo CPU.
    //**************Etapa 1: Remova o comentário das seguintes linhas, onde você está definindo a classificação dos dispositivos********
    /*if (dev.is_gpu() & (dev.get_info<info::device::name>().find(vendorName_) != std::string::npos))
        rating = 3;
    else if (dev.is_gpu()) rating = 2;
    else if (dev.is_cpu()) rating = 1; */
    return rating;
    };
    
private:
    std::string vendorName_;
};

// in_vect1 e in_vect2 são os vetores com num_elements, números complexos 
// e são entradas para a função paralela
void DpcppParallel(queue &q, std::vector<Complex2> &in_vect1,
                   std::vector<Complex2> &in_vect2, std::vector<Complex2> &out_vect) {
  // Configurar buffers de entrada
  buffer<Complex2, 1> bufin_vect1(in_vect1.data(), range<1>(num_elements));
  buffer<Complex2, 1> bufin_vect2(in_vect2.data(), range<1>(num_elements));

  // Configurar buffers de saída
  buffer<Complex2, 1> bufout_vect(out_vect.data(), range<1>(num_elements));

  std::cout << "Target Device: "
            << q.get_device().get_info<info::device::name>() <<"\n";
  // Enviar objeto de função do grupo Comando para a fila
  q.submit([&](handler &h) {
    // Acessores configurados em modo leitura
    auto V1 = bufin_vect1.get_access<access::mode::read>(h);
    auto V2 = bufin_vect2.get_access<access::mode::read>(h);
    // Acessor configurado em modo escrita
    //**************Etapa 2: Remova o comentário da linha abaixo para definir o acessor de escrita********************      
    //auto V3 = bufout_vect.get_access<access::mode::write>(h); 
      
    h.parallel_for(range<1>(num_elements), [=](id<1> i) {      
    //**************Etapa 3: Remova o comentário da linha abaixo para chamar a função complex_mul 
    //que calcula a multiplicação dos números complexos********************
        
    //V3[i] = V1[i].complex_mul(V2[i]); 
    });
  });
  q.wait_and_throw();
}
void DpcppScalar(std::vector<Complex2> &in_vect1, std::vector<Complex2> &in_vect2,
                 std::vector<Complex2> &out_vect) {
  for (int i = 0; i < num_elements; i++) {
    out_vect[i] = in_vect1[i].complex_mul(in_vect2[i]);
  }
}
//Compare os resultados dos dois vetores de saída de paralelo e escalar. Eles devem ser iguais
int Compare(std::vector<Complex2> &v1, std::vector<Complex2> &v2) {
  int ret_code = 1;
  for (int i = 0; i < num_elements; i++) {
    if (v1[i] != v2[i]) {
      ret_code = -1;
      break;
    }
  }
  return ret_code;
}
int main() {
  // Declare seus vetores de entrada e saída da classe Complex2
  vector<Complex2> input_vect1;
  vector<Complex2> input_vect2;
  vector<Complex2> out_vect_parallel;
  vector<Complex2> out_vect_scalar; 
  

  for (int i = 0; i < num_elements; i++) {
    input_vect1.push_back(Complex2(i + 2, i + 4));
    input_vect2.push_back(Complex2(i + 4, i + 6));
    out_vect_parallel.push_back(Complex2(0, 0));
    out_vect_scalar.push_back(Complex2(0, 0));
  }

  // este manipulador de exceção captura exceções assíncronas
  auto exception_handler = [&](cl::sycl::exception_list eList) {
    for (std::exception_ptr const &e : eList) {
      try {
        std::rethrow_exception(e);
      } catch (cl::sycl::exception const &e) {
        std::cout << "Failure" << std::endl;
        std::terminate();
      }
    }
  };

  // Inicialize seus vetores de entrada e saída. As entradas são inicializadas conforme abaixo.
  // As saídas são inicializadas com 0
  try {
    //Passe o nome do fornecedor para o qual o dispositivo que você deseja consultar
    std::string vendor_name = "Intel";
    //std::string vendor_name = "AMD";
    //std::string vendor_name = "Nvidia";
    // queue constructor passed exception handler
    custom_device_selector selector(vendor_name);
    queue q(selector, exception_handler);     
    // Chame DpcppParallel com as entradas e saídas necessárias
    DpcppParallel(q, input_vect1, input_vect2, out_vect_parallel);
  } catch (...) {
    // alguma outra exceção detectada
    std::cout << "Failure" << std::endl;
    std::terminate();
  }

  cout << "****************************************Multiplying Complex numbers "
          "in Parallel********************************************************"
       << std::endl;
  // Imprima as saídas da função Paralela
  for (int i = 0; i < num_elements; i++) {
    cout << out_vect_parallel[i] << ' ';
    if (i == num_elements - 1) {
      cout << "\n\n";
    }
  }
  cout << "****************************************Multiplying Complex numbers "
          "in Serial***********************************************************"
       << std::endl;
  // Chame a função DpcppScalar com as entradas e saídas necessárias
  DpcppScalar(input_vect1, input_vect2, out_vect_scalar);
  for (auto it = out_vect_scalar.begin(); it != out_vect_scalar.end(); it++) {
    cout << *it << ' ';
    if (it == out_vect_scalar.end() - 1) {
      cout << "\n\n";
    }
  }

  // Compare as saídas das funções paralela e escalar. Eles deveriam ser iguais
  int ret_code = Compare(out_vect_parallel, out_vect_scalar);
  if (ret_code == 1) {
    cout << "********************************************Success. Results are "
            "matched******************************"
         << "\n";
  } else
    cout << "*********************************************Failed. Results are "
            "not matched**************************"
         << "\n";

  return 0;
}

### Compilar e Executar
Selecione a célula abaixo e clique em executar ▶ para compilar e executar o código:

(Uma vez que é codificado para preferir uma GPU, os scripts são necessários para executar em um nó com uma GPU.)

In [ ]:
! chmod 755 q; chmod 755 run_complex_mult.sh;if [ -x "$(command -v qsub)" ]; then ./q run_complex_mult.sh; else ./run_complex_mult.sh; fi

Se as células do Jupyter não responderem ou se apresentarem um erro ao compilar os exemplos de código, reinicie o kernel do Jupyter. Selecione o seguinte item de menu na janela Jupyter: `Kernel > Restart Kernel and Clear All Outputs` e compile os exemplos de código novamente.

#### Solução
- [complex_mult_solution.cpp](src/complex_mult_solution.cpp)

***
# Resumo

Neste módulo você aprendeu:
* As classes SYCL fundamentais
* Como selecionar o dispositivo para descarregar nas cargas de trabalho do kernel
* Como escrever um programa SYCL usando _buffers_, acessores, manipulador de grupo de comando e _kernel_
* Como usar os acessores de host e destruição de _buffer_ para fazer a sincronização